<a href="https://colab.research.google.com/github/Eun0503/safecall/blob/main/%EC%95%88%EC%8B%AC%ED%86%B5%ED%99%94%EC%A7%80%ED%82%B4%EC%9D%B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install evaluate>=0.30
!pip install jiwer
!pip install accelerate -U
!pip install transformers[torch]

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-wg8p774c
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-wg8p774c
  Resolved https://github.com/huggingface/transformers to commit 15c74a28294fe9082b81b24efe58df16fed79a9e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.41.0.dev0-py3-none-any.whl size=9093626 sha256=5456106f8acab318f0afd82bd40d48e5d588e751b4cf163d605fd8753853ad80
  Stored in directory: /tmp/pip-ephem-wheel-cache-8ztk_vhq/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.2
    Uninstalling transformers-4.40.2:
      Successfully uninstalled transformers-4.40.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from transformers import WhisperFeatureExtractor
# 파인튜닝을 진행하고자 하는 모델의 feature extractor를 로드
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [3]:
from transformers import WhisperTokenizer
# 파인튜닝을 진행하고자 하는 모델의 tokenizer를 로드
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-base", language="Korean", task="transcribe")

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
input_str = "저는 서울중앙지검 지능범죄수사팀 최인호 검사입니다."
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")

Input:                 저는 서울중앙지검 지능범죄수사팀 최인호 검사입니다.
Decoded w/ special:    <|startoftranscript|><|ko|><|transcribe|><|notimestamps|>저는 서울중앙지검 지능범죄수사팀 최인호 검사입니다.<|endoftext|>
Decoded w/out special: 저는 서울중앙지검 지능범죄수사팀 최인호 검사입니다.
Are equal:             True


In [5]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-base", language="Korean", task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [20]:
# 1. 오디오 파일 경로 취합
import glob

path = "/content/drive/MyDrive/안심통화지킴이데이터셋/TS_D03/*/*/*/*"
raw_data_list = glob.glob(path)
raw_data_list = sorted(raw_data_list)
print(f"file_list : {raw_data_list[:10]}")
print(len(raw_data_list))

file_list : ['/content/drive/MyDrive/안심통화지킴이데이터셋/TS_D03/D03/J13/S000002/0002.wav', '/content/drive/MyDrive/안심통화지킴이데이터셋/TS_D03/D03/J13/S000002/0003.wav', '/content/drive/MyDrive/안심통화지킴이데이터셋/TS_D03/D03/J13/S000002/0004.wav', '/content/drive/MyDrive/안심통화지킴이데이터셋/TS_D03/D03/J13/S000002/0005.wav', '/content/drive/MyDrive/안심통화지킴이데이터셋/TS_D03/D03/J13/S000002/0006.wav', '/content/drive/MyDrive/안심통화지킴이데이터셋/TS_D03/D03/J13/S000002/0007.wav', '/content/drive/MyDrive/안심통화지킴이데이터셋/TS_D03/D03/J13/S000002/0008.wav', '/content/drive/MyDrive/안심통화지킴이데이터셋/TS_D03/D03/J13/S000002/0009.wav', '/content/drive/MyDrive/안심통화지킴이데이터셋/TS_D03/D03/J13/S000002/0010.wav', '/content/drive/MyDrive/안심통화지킴이데이터셋/TS_D03/D03/J13/S000002/0011.wav']
2000


In [21]:
# 2. 텍스트 파일 경로 취합
path = "/content/drive/MyDrive/안심통화지킴이데이터셋/TL_D03/*/*/*/*"
labeled_data_list = glob.glob(path)
# 레이블 데이터에는 json 데이터가 폴더별로 하나씩 있으므로 txt 파일만을 골라낸다.
labeled_data_list = sorted([file for file in labeled_data_list if file.endswith(".txt")])
print(f"file_list : {labeled_data_list[:10]}")
print(len(labeled_data_list))

file_list : ['/content/drive/MyDrive/안심통화지킴이데이터셋/TL_D03/D03/J13/S000002/0002.txt', '/content/drive/MyDrive/안심통화지킴이데이터셋/TL_D03/D03/J13/S000002/0003.txt', '/content/drive/MyDrive/안심통화지킴이데이터셋/TL_D03/D03/J13/S000002/0004.txt', '/content/drive/MyDrive/안심통화지킴이데이터셋/TL_D03/D03/J13/S000002/0005.txt', '/content/drive/MyDrive/안심통화지킴이데이터셋/TL_D03/D03/J13/S000002/0006.txt', '/content/drive/MyDrive/안심통화지킴이데이터셋/TL_D03/D03/J13/S000002/0007.txt', '/content/drive/MyDrive/안심통화지킴이데이터셋/TL_D03/D03/J13/S000002/0008.txt', '/content/drive/MyDrive/안심통화지킴이데이터셋/TL_D03/D03/J13/S000002/0009.txt', '/content/drive/MyDrive/안심통화지킴이데이터셋/TL_D03/D03/J13/S000002/0010.txt', '/content/drive/MyDrive/안심통화지킴이데이터셋/TL_D03/D03/J13/S000002/0011.txt']
2000


In [22]:
from tqdm import tqdm
import pandas as pd

In [23]:
transcript_list = []
for labeled_data in tqdm(labeled_data_list):
    with open(labeled_data, 'r', encoding='UTF8') as f:
        line = f.readline()
        transcript_list.append(line)

df = pd.DataFrame(data=transcript_list, columns = ["transcript"])

# 텍스트 데이터로 만든 데이터프레임에 음성 파일 경로 컬럼을 추가
df["raw_data"] = raw_data_list

100%|██████████| 2000/2000 [22:50<00:00,  1.46it/s]


In [24]:
# Null data 유무 확인
df.isnull().values.sum()

0

In [47]:
!pip install huggingface_hub
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write)

In [50]:
from datasets import Dataset, DatasetDict
from datasets import Audio

# 오디오 파일 경로를 dict의 "audio" 키의 value로 넣고 이를 데이터셋으로 변환
# 이때, Whisper가 요구하는 사양대로 Sampling rate는 16,000으로 설정한다.
ds = Dataset.from_dict({"audio": [path for path in df["raw_data"]],
                       "transcripts": [transcript for transcript in df["transcript"]]}).cast_column("audio", Audio(sampling_rate=16000))

# 데이터셋을 훈련 데이터와 테스트 데이터, 밸리데이션 데이터로 분할
train_testvalid = ds.train_test_split(test_size=0.2)
test_valid = train_testvalid["test"].train_test_split(test_size=0.5)
datasets = DatasetDict({
    "train": train_testvalid["train"],
    "test": test_valid["test"],
    "valid": test_valid["train"]})

# 작성한 데이터셋을 허깅페이스에 업로드
datasets.push_to_hub("Eun0503/safecall")



Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/16 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Eun0503/safecall/commit/624949ea2469eeb7140bd9757074861980db92fb', commit_message='Upload dataset', commit_description='', oid='624949ea2469eeb7140bd9757074861980db92fb', pr_url=None, pr_revision=None, pr_num=None)

In [51]:
from datasets import load_dataset
# 데이터셋을 업로드할 때 접근을 제한하거나 비공개로 설정한 경우 허깅페이스 로그인이 필요하다.
low_call_voices  = load_dataset("Eun0503/safecall")

Generating train split:   0%|          | 0/1600 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/200 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/200 [00:00<?, ? examples/s]

In [52]:
def prepare_dataset(batch):
    # 오디오 파일을 16kHz로 로드
    audio = batch["audio"]

    # input audio array로부터 log-Mel spectrogram 변환
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # target text를 label ids로 변환
    batch["labels"] = tokenizer(batch["transcripts"]).input_ids
    return batch

In [53]:
# 데이터 전처리 함수를 데이터셋 전체에 적용
low_call_voices = datasets.map(prepare_dataset, remove_columns=datasets.column_names["train"], num_proc=None)

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [54]:
# 전처리 작업이 오래 걸릴 수 있으므로, colab을 사용하여 파인튜닝을 진행한다면 전처리가 완료된 데이터셋을 Hub에 저장하는 것을 추천한다.
low_call_voices.push_to_hub("Eun0503/safecall-preprocessed")

Uploading the dataset shards:   0%|          | 0/4 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Eun0503/safecall-preprocessed/commit/c983b629d3372ce158ab0c14f810654f33c63027', commit_message='Upload dataset', commit_description='', oid='c983b629d3372ce158ab0c14f810654f33c63027', pr_url=None, pr_revision=None, pr_num=None)

In [55]:
# Hub로부터 전처리가 완료된 데이터셋을 로드
from datasets import load_dataset
low_call_voices_prepreocessed = load_dataset("Eun0503/safecall-preprocessed")
low_call_voices_prepreocessed

Generating train split:   0%|          | 0/1600 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/200 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/200 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 1600
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 200
    })
    valid: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 200
    })
})

In [56]:
import torch
from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # 인풋 데이터와 라벨 데이터의 길이가 다르며, 따라서 서로 다른 패딩 방법이 적용되어야 한다. 그러므로 두 데이터를 분리해야 한다.
        # 먼저 오디오 인풋 데이터를 간단히 토치 텐서로 반환하는 작업을 수행한다.
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # Tokenize된 레이블 시퀀스를 가져온다.
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # 레이블 시퀀스에 대해 최대 길이만큼 패딩 작업을 실시한다.
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # 패딩 토큰을 -100으로 치환하여 loss 계산 과정에서 무시되도록 한다.
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # 이전 토크나이즈 과정에서 bos 토큰이 추가되었다면 bos 토큰을 잘라낸다.
        # 해당 토큰은 이후 언제든 추가할 수 있다.
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [57]:
# 훈련시킬 모델의 processor, tokenizer, feature extractor 로드
from transformers import WhisperTokenizer,  WhisperFeatureExtractor
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-base", language="Korean", task="transcribe")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-base", language="Korean", task="transcribe")
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-base")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [58]:
# 데이터 콜레이터 초기화
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [59]:
import evaluate

metric = evaluate.load('cer')

In [60]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # pad_token을 -100으로 치환
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # metrics 계산 시 special token들을 빼고 계산하도록 설정
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    cer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"cer": cer}

In [61]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base")

config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.81k [00:00<?, ?B/s]

In [62]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [68]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="safecall-model",  # 원하는 리포지토리 이름을 임력한다.
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # 배치 크기가 2배 감소할 때마다 2배씩 증가
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,  # epoch 대신 설정
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="cer",  # 한국어의 경우 'wer'보다는 'cer'이 더 적합할 것
    greater_is_better=False,
    push_to_hub=True,
)


ValueError: FP16 Mixed precision training with AMP or APEX (`--fp16`) and FP16 half precision evaluation (`--fp16_full_eval`) can only be used on CUDA or MLU devices or NPU devices or certain XPU devices (with IPEX).